# Extension to influenza

This notebook simulates the epidemic models using an Influenza parameterisation. The steps are identical to the COVID-19 simulation.

In [3]:
# to allow relative imports
import os
from sys_path_util import append_sys_path
append_sys_path()

# experiment
from epyc import Lab, JSONLabNotebook, RepeatedExperiment, ParallelLab

# generators
from lib.model.network.mobility_network import MNGeneratorFromNetworkData as MNG

# file utils
from lib.experiments.utils.data_repo_api import DataRepoAPI
from lib.experiments.utils.network_data_utils import load_network_data_from_files

# other utils
from functools import partial
import numpy as np
from mpmath import polylog
from datetime import datetime as dt
import pickle

# multiprocessing
from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [4]:
# Some global parameters

N = 10000
CUTOFF = 40
MOBILITY_EXPONENT = 2
n_exp = 10

# Experiment result output dir
output_dir = 'experiment_results'
if not output_dir in os.listdir():
    os.mkdir(output_dir)

def plc_mean(exponent, cutoff):
    """
    Calculate theoretical mean of a power law with cutoff distribution from
    the exponent and the cutoff.
    """
    nom = polylog(exponent-1, np.exp(-1 / cutoff))
    denom = polylog(exponent, np.exp(-1 / cutoff))
    return nom / denom

def estimate_exponent_for_mean(mean, cutoff, tolerance):
    """
    Given the mean and cutoff of a power law with cutoff distribution,
    estimate the corresponding exponent with a given tolerance.
    """
    grid = np.arange(1, 5, 0.01)
    diffs = []
    for i in range(len(grid)):
        diff = abs(mean - plc_mean(grid[i], cutoff))
        if diff < tolerance:
            return grid[i]
        diffs.append(diff)
    
    return grid[diffs.index(min(diffs))]

## Network

In [5]:
# Load the network data

file_names = dict(
    demographics='demographics.pkl',
    comb_pre='comb_counts_pre.pkl',
    comb_post='comb_counts_post.pkl',
    trip_pre='trip_counts_pre.pkl',
    trip_post='trip_counts_post.pkl',
)

network_data = load_network_data_from_files(file_names)

network_data_pre = network_data['pre']
network_data_post = network_data['post']

In [6]:
# Define the generator parameters
params_mobility_pre = dict()
params_mobility_pre[MNG.N] = N
params_mobility_pre[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_pre[MNG.CUTOFF] = CUTOFF
params_mobility_pre[MNG.MULTIPLIER] = False

params_mobility_post = dict()
params_mobility_post[MNG.N] = N
params_mobility_post[MNG.EXPONENT] = MOBILITY_EXPONENT
params_mobility_post[MNG.CUTOFF] = CUTOFF
params_mobility_post[MNG.MULTIPLIER] = True

In [7]:
# Generator instances
mng_pre = MNG(network_data=network_data_pre)
mng_post = MNG(network_data=network_data_post)

## Epidemic Models

In [ ]:
# List to store all the experiment runners
expms = list()

In [ ]:
# Import models etc
from epydemic import SEIR
from lib.model.compartmental_model.seir import MonitoredSEIR
from lib.model.compartmental_model.seir import SEIRWithQuarantine as SEIRQ
from lib.model.compartmental_model.seir import MonitoredSEIRWithQuarantine as MonitoredSEIRQ
from lib.model.compartmental_model.seivr import SEIVR, MonitoredSEIVR
from lib.model.compartmental_model.seivr import SEIVRWithQuarantine as SEIVRQ
from lib.model.compartmental_model.seivr import MonitoredSEIVRWithQuarantine as MonitoredSEIVRQ

from epydemic import StochasticDynamics, Monitor

from lib.model.distributions import PowerLawCutoffDist as PLCD

In [ ]:
# Helper functions

def calc_p_infect(r0, p_remove, k_mean, k_var):
    """
    Calc p_infect.
    :param r0: basic reproduction number
    :param p_remove: probability of removal
    :param k_mean: Mean of node degree distribution
    :param k_var: Variance of node degree distribution
    :return: p_infect
    """
    return float(p_remove * r0 * (k_mean / (k_var - k_mean)))


def run_stochastic_dynamics_lab(model, generator, params, 
                                file_name, description, n=n_exp,
                                output_dir=output_dir, push_to_repo=True,
                                T_max=300):
    """
    Run a StochasticDynamics experiment repeatadly in a lab.
    """

    file = os.path.join(output_dir, file_name)
    
    nb = JSONLabNotebook(file, create=True, description=description)
    lab = ParallelLab(notebook=nb, cores=min(cpu_count()-1, n))
    
    e = StochasticDynamics(model, generator)
    e.process().setMaximumTime(T_max)

    # pass params to the lab
    for k, v in params.items():
        lab[k] = v

    lab.runExperiment(RepeatedExperiment(e, n))
    
    if push_to_repo:
        DataRepoAPI.update_or_create(
            file_name=file_name,
            file_path=output_dir,
            repo_path='simulations/influenza'
        )
    
    return lab.results()

### Parameters

In [ ]:
# Fixed
R0 = 1.46  # Biggerstaff et al. 2014
_P_INFECT = None  # to calculate later 
P_REMOVE = 0.1428  # 7 days, Ashleigh et al. 2010
P_SYMPTOMS = 0.25  # 4 days, Ashleigh et al. 2010
P_EXPOSED = 0.01

_RRR = [0.5, 0.65, 0.8, 0.95]

# Free
_P_VACCINATED = [0.001, 0.004, 0.007, 0.01]
_P_VACCINATED_INITAL = [0.0, 0.2, 0.4, 0.6, 0.8]
_P_QUARANTINE = [0.0, 0.25, 0.5, 0.75, 1]

In [ ]:
# Fix parameters as far as possible

params_seir = dict()
params_seir[SEIR.P_SYMPTOMS] = P_SYMPTOMS
params_seir[SEIR.P_REMOVE] = P_REMOVE
params_seir[SEIR.P_EXPOSED] = P_EXPOSED

params_seivr = dict()
params_seivr[SEIVR.P_SYMPTOMS] = P_SYMPTOMS
params_seivr[SEIVR.P_REMOVE] = P_REMOVE
params_seivr[SEIVR.P_EXPOSED] = P_EXPOSED

params_monitor = dict()
params_monitor[Monitor.DELTA] = 10

In [ ]:
# Calculate P_INFECT

# Mobility
plcd = PLCD(MOBILITY_EXPONENT, CUTOFF)
P_INFECT_MOBILITY = calc_p_infect(R0, P_REMOVE, plcd.mean, plcd.var)

#### SEIR

In [ ]:
# params
params_pre1 = {**params_mobility_pre, **params_seir, **params_monitor}
params_pre1[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre1[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

params_post1 = {**params_mobility_post, **params_seir, **params_monitor}
params_post1[SEIR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post1[SEIR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2

In [ ]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(),
    mng_pre, 
    params_pre1, 
    'influenza_seir_mobility_pre.json',
    'SEIR simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIR(), 
    mng_post, 
    params_post1, 
    'influenza_seir_mobility_post.json',
    'SEIR simulation with Mobility Network (Post)'
))

### SEIR_Q

In [ ]:
params_pre2 = {**params_mobility_pre, **params_seir, **params_monitor}
params_pre2[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre2[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre2[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

params_post2 = {**params_mobility_post, **params_seir, **params_monitor}
params_post2[SEIRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post2[SEIRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post2[SEIRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_pre, 
    params_pre2, 
    'influenza_seirq_mobility_pre.json',
    'SEIR_Q simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIRQ(), 
    mng_post, 
    params_post2, 
    'influenza_seirq_mobility_post.json',
    'SEIR_Q simulation with Mobility Network (Post)'
))

### SEIVR

In [ ]:
params_pre7 = {**params_mobility_pre, **params_seivr, **params_monitor}
params_pre7[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre7[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre7[SEIVR.P_VACCINATED] = _P_VACCINATED
params_pre7[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre7[SEIVR.VACCINE_RRR] = _RRR

params_post7 = {**params_mobility_post, **params_seivr, **params_monitor}
params_post7[SEIVR.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post7[SEIVR.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post7[SEIVR.P_VACCINATED] = _P_VACCINATED
params_post7[SEIVR.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post7[SEIVR.VACCINE_RRR] = _RRR

In [ ]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_pre, 
    params_pre7, 
    'influenza_seivr_mobility_pre.json',
    'SEIVR simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVR(), 
    mng_post, 
    params_post7, 
    'influenza_seivr_mobility_post.json',
    'SEIVR simulation with Mobility Network (Post)'
))

### SEIVR_Q

In [ ]:
params_pre8 = {**params_mobility_pre, **params_seivr, **params_monitor}
params_pre8[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_pre8[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_pre8[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_pre8[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_pre8[SEIVRQ.VACCINE_RRR] = _RRR
params_pre8[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

params_post8 = {**params_mobility_post, **params_seivr, **params_monitor}
params_post8[SEIVRQ.P_INFECT_SYMPTOMATIC] = P_INFECT_MOBILITY
params_post8[SEIVRQ.P_INFECT_ASYMPTOMATIC] = P_INFECT_MOBILITY / 2
params_post8[SEIVRQ.P_VACCINATED] = _P_VACCINATED
params_post8[SEIVRQ.P_VACCINATED_INITIAL] = _P_VACCINATED_INITAL
params_post8[SEIVRQ.VACCINE_RRR] = _RRR
params_post8[SEIVRQ.P_QUARANTINE] = _P_QUARANTINE

In [ ]:
expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_pre, 
    params_pre8, 
    'influenza_seivrq_mobility_pre.json',
    'SEIVR_Q simulation with Mobility Network (Pre)',
))

expms.append(lambda: run_stochastic_dynamics_lab(
    MonitoredSEIVRQ(), 
    mng_post, 
    params_post8, 
    'influenza_seivrq_mobility_post.json',
    'SEIVR_Q simulation with Mobility Network (Post)'
))

## Run experiments ...

In [ ]:
# %%time
# num_cores = min(len(expms), cpu_count()-1, 12)
# print(f"Running {num_cores} jobs on {cpu_count()} available cores...")
# print("Start time %s" % dt.now())
# with Parallel(n_jobs=num_cores) as processes:
#     procs = processes(delayed(e)() for e in expms)
# print("End time %s" % dt.now())

for expm in expms:
    print("Start time: %s" % dt.now())
    expm()
    print("End time: %s" % dt.now())